In [1]:
import pandas as pd
import os
import numpy as np
import cv2
from tqdm import tqdm
from glob import glob
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from imblearn.over_sampling import SMOTE
from timm import create_model

In [2]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [5]:
_HYPER_KVASIR = 'D:\\data\\endoscopic\\hyper_kvasir'
# _HYPER_KVASIR_LABELED_DEFORM = 'D:\\data\\endoscopic\\hyper_kvasir\\labeled_images_deform'

In [8]:
df_anno = pd.read_csv('../resource/hyper_kvasir/full_labeled_anno.csv')
df_anno.head()

,Organ,Classification,Finding,Image,Categories,is_valid
0,upper-gi-tract,anatomical-landmarks,z-line,upper-gi-tract/anatomical-landmarks/z-line/c7e...,0,False
1,lower-gi-tract,therapeutic-interventions,dyed-lifted-polyps,lower-gi-tract/therapeutic-interventions/dyed-...,0,False
2,lower-gi-tract,quality-of-mucosal-views,bbps-2-3,lower-gi-tract/quality-of-mucosal-views/bbps-2...,0,False
3,lower-gi-tract,quality-of-mucosal-views,bbps-2-3,lower-gi-tract/quality-of-mucosal-views/bbps-2...,0,False
4,upper-gi-tract,anatomical-landmarks,z-line,upper-gi-tract/anatomical-landmarks/z-line/b98...,0,False


In [19]:
# df_anno['Categories'].value_counts()

In [9]:
list_cat_abn = []
for c in df_anno['Categories']:
    if c != 0:
        list_cat_abn.append(1)
    else:
        list_cat_abn.append(c)
df_anno['Abnormal'] = list_cat_abn

In [10]:
df_anno['Abnormal'].value_counts()

0    8020
1    2642
Name: Abnormal, dtype: int64

In [22]:
x_train, x_valid, y_train, y_valid = train_test_split(df_anno['Image'], df_anno['Abnormal'], test_size=0.2, random_state=0)
df_train = pd.DataFrame([])
df_train['image'] = x_train
df_train['target'] = y_train
df_train['is_valid'] = False

df_valid = pd.DataFrame([])
df_valid['image'] = x_valid
df_valid['target'] = y_valid
df_valid['is_valid'] = True

df_full = pd.concat([df_train, df_valid], axis=0).reset_index()
df_full

,index,image,target,is_valid
0,7868,lower-gi-tract/quality-of-mucosal-views/bbps-2...,0,False
1,8674,lower-gi-tract/quality-of-mucosal-views/bbps-2...,0,False
2,2923,lower-gi-tract/anatomical-landmarks/cecum/12fc...,0,False
3,4818,lower-gi-tract/quality-of-mucosal-views/bbps-2...,0,False
4,6224,lower-gi-tract/quality-of-mucosal-views/impact...,0,False
...,...,...,...,...
10657,7321,upper-gi-tract/anatomical-landmarks/retroflex-...,0,True
10658,2578,lower-gi-tract/therapeutic-interventions/dyed-...,0,True
10659,4136,lower-gi-tract/quality-of-mucosal-views/bbps-2...,0,True
10660,7908,lower-gi-tract/therapeutic-interventions/dyed-...,0,True


In [11]:
# x = df_anno['path']
# y = df_anno['Categories']

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
# x_train_lab, x_train_unlab, y_train_lab, y_train_unlab = train_test_split(x_train, 
#                                                                           y_train, 
#                                                                           test_size=0.32, 
#                                                                           random_state=0)


# df_train_lab = pd.DataFrame([])
# df_train_lab = pd.DataFrame([])
# df_train_lab['image'] = x_train_lab
# df_train_lab['target'] = y_train_lab
# df_train_lab['is_valid'] = False
# df_train_lab['is_labeled'] = True

# df_train_unlab = pd.DataFrame([])
# df_train_unlab = pd.DataFrame([])
# df_train_unlab['image'] = x_train_unlab
# df_train_unlab['target'] = y_train_unlab
# df_train_unlab['is_valid'] = False
# df_train_unlab['is_labeled'] = False


# df_valid['image'] = x_test
# df_valid['target'] = y_test
# df_valid['is_valid'] = True
# df_valid['is_labeled'] = True

In [12]:
# df_full = pd.concat([df_train_lab, df_train_unlab, df_valid], axis=0).reset_index()
# del df_full['index']
# df_full

In [13]:
# fig = df_full['target'].value_counts().plot(kind='bar', figsize=(10, 8), fontsize=10, rot=0).get_figure()

In [23]:
df_full.to_csv('../resource/hyper_kvasir/df_abnormal.csv', index=False, header=True)

In [15]:
# cnt_train = df_train['target'].value_counts()
# cnt_valid = df_valid['target'].value_counts()

# df_cnt = pd.DataFrame({'train': cnt_train,'valid': cnt_valid})
# chart = df_cnt.plot.bar(rot=0, figsize=(10, 8), fontsize=15, stacked=True).get_figure()
# # chart.savefig('../resource/visualize/visual_sup.png')

In [69]:
# cnt_train = df_train['Groupby_Categories']
# cnt_train_labeled, cnt_train_unlabeled = train_test_split(cnt_train, test_size = 0.9, random_state = 0)
# cnt_train_labeled, cnt_train_unlabeled = cnt_train_labeled.value_counts(), cnt_train_unlabeled.value_counts()
# cnt_valid = df_valid['Groupby_Categories'].value_counts()

# df_cnt = pd.DataFrame({'train_labeled': cnt_train_labeled, 
                       
#                        'train_unlabeled': cnt_train_unlabeled,
#                        'valid': cnt_valid,
#                        })
# chart_semi = df_cnt.plot.bar(rot=0, 
#                         figsize=(10, 8), 
#                         fontsize=15,
#                         color={"train_labeled": "#1f76b5", "train_unlabeled": "green", "valid":"#fe7e0e"},
#                         stacked=True).get_figure()
# chart_semi.savefig('../resource/visualize/visual_semi_1_9.png')

In [70]:
# cnt_train_labeled, cnt_train_unlabeled

In [71]:
# df_train['Groupby_Categories'].value_counts()
# fig_train.savefig('../resource/visualize/groupcat_train.png')

In [72]:
# df_valid['Groupby_Categories'].value_counts()
# fig_valid.savefig('../resource/visualize/groupcat_valid.png')

In [14]:
# fig.savefig('../resource/visualize/groupcat.png')

In [15]:
# df_full.to_csv('../resource/hyper_kvasir/full_labeled_group.csv', index=False, header=True)